# Parallel and Distributed Computing

In [1]:
function calculate_pi(n)
    inside = 0
    for i = 1:n
        x = rand()
        y = rand()
        inside += (x^2 + y^2) <= 1.0 ? 1 : 0
    end
    return 4 * inside / n
end

calculate_pi (generic function with 1 method)

In [2]:
calculate_pi(1e10)

3.1415561152

In [3]:
using Distributed

# Add worker processes equal to the number of available cores
addprocs(Sys.CPU_THREADS)


4-element Vector{Int64}:
 2
 3
 4
 5

In [4]:
@everywhere function calculate_pi(n)
    inside = 0
    for i = 1:n
        x = rand()
        y = rand()
        inside += (x^2 + y^2) <= 1.0 ? 1 : 0
    end
    return 4 * inside / n
end

In [5]:
using Statistics

# Each worker will calculate π using n points
n = 2.5 * 10^9
pies = @distributed (vcat) for i in 1:nworkers()
    calculate_pi(n)
end

pi_estimate = mean(pies)

println("Estimate of π: $pi_estimate")


Estimate of π: 3.1415866864


## GPU Programming

In [9]:
import Pkg
Pkg.add("Metal")

   Resolving package versions...


   Installed XZ_jll ─────────────── v5.4.5+0
   Installed UnsafeAtomicsLLVM ──── v0.1.3
   Installed TimerOutputs ───────── v0.5.23


   Installed Metal_LLVM_Tools_jll ─ v0.5.1+0
   Installed ExprTools ──────────── v0.1.10
   Installed Python_jll ─────────── v3.10.8+1
   Installed GPUArrays ──────────── v9.1.0


   Installed LLVMExtra_jll ──────── v0.0.27+1
   Installed GPUArraysCore ──────── v0.1.5
   Installed UnsafeAtomics ──────── v0.2.1
   Installed KernelAbstractions ─── v0.9.13
   Installed Atomix ─────────────── v0.1.0


   Installed SQLite_jll ─────────── v3.43.0+0
   Installed LLVM ───────────────── v6.4.0
   Installed GPUCompiler ────────── v0.24.5


...

  ✓ UnsafeAtomicsLLVM
  ✓ GPUArrays


  ✓ KernelAbstractions


  ✓ GPUCompiler


  ✓ Metal


  ✓ Metal → SpecialFunctionsExt
  22 dependencies successfully precompiled in 32 seconds. 321 already precompiled.


In [14]:
using Metal

function pi_approximation_kernel(xs, ys, inside)
    i = thread_position_in_grid_1d()
    x = xs[i]
    y = ys[i]
    if x^2 + y^2 <= 1.0
        @inbounds inside[i] = 1
    else
        @inbounds inside[i] = 0
    end
    return
end

# Use a large number of points
n = 10^10

# Create arrays on the GPU
xs = MtlArray(rand(Float32, n))
ys = MtlArray(rand(Float32, n))
inside = MtlArray(zeros(Int32, n))

# Define thread configuration
threads_per_group = 256
num_groups = ceil(Int, n / threads_per_group)

# Launch the kernel
@metal threads=threads_per_group groups=num_groups pi_approximation_kernel(xs, ys, inside)

# Copy the result back to the CPU and calculate π
inside_cpu = Array(inside)
pi_estimate = 4 * sum(inside_cpu) / n

println("Estimate of π: $pi_estimate")